In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

memory_chat = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=memory_chat)

def add_message(input, output) :
    memory.save_context({"input":input}, {"output":output})

def get_summary():
    return memory.load_memory_variables({})

add_message("Hi my name is KKY, I live in South Korea", "WOW that is so cool")

In [6]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [7]:
get_summary()

{'history': 'The human introduces themselves as KKY from South Korea. The AI expresses excitement and finds it cool, wishing it could visit South Korea because it is so pretty.'}